In [ ]:
import gymnasium as gym
import numpy as np
import torch
from psutil import cpu_count
from pydrake.all import StartMeshcat

#import manipulation.envs.box_flipup  # no-member
import Franka_gym_pos
from Franka_gym_pos import *
from manipulation.utils import FindDataResource, RenderDiagram, running_as_notebook

num_cpu = int(cpu_count() / 2) if running_as_notebook else 2

In [ ]:
meshcat = StartMeshcat()

In [ ]:
env = gym.make("Franka-v1", meshcat=meshcat)
obs, _ = env.reset()
env.render()

for i in range(50):

    test = [int(input("Coose an action (0, 1 or 2) :"))]

    obs, reward, terminated, truncated, info= env.step(test)
    #print(obs)
    env.render()